In [ ]:
#Final Verison
# Used 10min2s to retrieve Jan to May (5 months) data
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
def get_url(soup):
  div_tags = soup.find_all("div", class_="item")
  links = []
  for div_tag in div_tags:
    span_tag = div_tag.find("span", class_="title")
    link = span_tag.find("a")["href"]
    links.append(link)
  basic_url= "https://news.rthk.hk" #Combine the links together
  target_url=[]
  for target_link in links:
    target_url.append(basic_url+target_link)
  return target_url
def news_extract(target_url):
  news_df = pd.DataFrame(columns=['Title', 'Content',"Date"])
  news_response= requests.get(target_url)
  news_soup = BeautifulSoup(news_response.content, "html.parser")
  news_title= news_soup.find("h2", class_="itemTitle").text.strip()
  news_content= news_soup.find("div",class_="itemFullText").text.strip()
  news_date= news_soup.find("div",class_="createddate").text.strip()
  news_row= pd.DataFrame({"Title":[news_title],"Content":[news_content],"Date":[news_date]})
  news_df= pd.concat([news_df,news_row], ignore_index=True)
  return news_df
def check_empty(soup):
  content = False
  check_content= "There is no News according to your request."
  div_tags = soup.find_all("div", class_="item")
  for div_tag in div_tags:
    span_tag = div_tag.find("span")
    if span_tag.text == check_content:
        content = False
    else:
        content= True
        break
  return content
year=2022 # Modify Time
# month=5
# day=31
for month in range(8,13):
  news_file = pd.DataFrame(columns=['Title', 'Content',"Date"])
  for day in range(1,31):
    archive_url= f"https://news.rthk.hk/rthk/en/news-archive.htm?archive_year={year}&archive_month=0{month}&archive_day={day}&archive_cat=12" # archive_cat=12-> finance
    session= requests.session()
    response=session.get(archive_url)
    soup = BeautifulSoup(response.content, "html.parser")
    if check_empty(soup)== False:
      pass
    else:
      for news_url in get_url(soup):
        news_file = pd.concat([news_file,news_extract(news_url)],ignore_index=True)
  news_file.to_csv(f"{year}{month:02d}Financial News from RTHK.csv")

